# Train network notebook

This notebook is coded to help train a U-net using atrous (dilated) convolutions.
It can be used either in GUI mode with jupyter or in console mode (TO CHECK).

First, we import the modules.
We also import a set of configuration functions that help us when manipulation config files, which are the backbone of the network.

In [1]:
import json
import os
import time
from AxonDeepSeg.config_tools import *

We define the name of the directory we are going to save our model to.

In [2]:
dir_model_save='model_christian_0'

We also define the configuration of the model.

In [3]:
default_configuration()

{'balanced_weights': [1.1, 1, 1.3],
 'batch_norm_activate': True,
 'batch_norm_decay_decay_activate': True,
 'batch_norm_decay_decay_period': 24000,
 'batch_norm_decay_ending_decay': 0.9,
 'batch_norm_decay_starting_decay': 0.7,
 'batch_size': 8,
 'convolution_per_layer': [3, 3, 3, 3],
 'da-elastic-activate': True,
 'da-elastic-alpha_max': 9,
 'da-elastic-order': 3,
 'da-flipping-activate': True,
 'da-flipping-order': 4,
 'da-gaussian_blur-activate': True,
 'da-gaussian_blur-order': 6,
 'da-gaussian_blur-sigma_max': 1.5,
 'da-noise_addition-activate': False,
 'da-noise_addition-order': 5,
 'da-random_rotation-activate': False,
 'da-random_rotation-high_bound': 89,
 'da-random_rotation-low_bound': 5,
 'da-random_rotation-order': 2,
 'da-rescaling-activate': False,
 'da-rescaling-factor_max': 1.2,
 'da-rescaling-order': 1,
 'da-shifting-activate': True,
 'da-shifting-order': 0,
 'da-shifting-percentage_max': 0.1,
 'da-type': 'all',
 'dataset_mean': 120.95,
 'dataset_variance': 60.23,
 'd

# Dictionnary with the updates
update_dict={
    'dilation_rate': [[1, 1, 1], [1, 1, 1], [1,1,1,1,1,1]],
    'convolution_per_layer': [3, 3, 6],
    'features_per_convolution': [[[1, 16], [16, 16], [16, 16]],
  [[16, 32], [32, 32], [32, 32]],
  [[32, 64], [64, 64], [64, 64],[64, 128], [128, 128], [128, 128]]],
    'size_of_convolutions_per_layer': [[5, 5, 5],
  [3, 3, 3],
  [3, 3, 3, 3, 3, 3]]
}


config=update_config(default_configuration(),update_dict)

In [4]:
# Dictionnary with the updates
update_dict={'batch_size':2,
             'learning_rate_decay_period': 48000,
             'batch_norm_decay_decay_period': 72000
}


config=update_config(default_configuration(),update_dict)

In [5]:
config

{'balanced_weights': [1.1, 1, 1.3],
 'batch_norm_activate': True,
 'batch_norm_decay_decay_activate': True,
 'batch_norm_decay_decay_period': 72000,
 'batch_norm_decay_ending_decay': 0.9,
 'batch_norm_decay_starting_decay': 0.7,
 'batch_size': 2,
 'convolution_per_layer': [3, 3, 3, 3],
 'da-elastic-activate': True,
 'da-elastic-alpha_max': 9,
 'da-elastic-order': 3,
 'da-flipping-activate': True,
 'da-flipping-order': 4,
 'da-gaussian_blur-activate': True,
 'da-gaussian_blur-order': 6,
 'da-gaussian_blur-sigma_max': 1.5,
 'da-noise_addition-activate': False,
 'da-noise_addition-order': 5,
 'da-random_rotation-activate': False,
 'da-random_rotation-high_bound': 89,
 'da-random_rotation-low_bound': 5,
 'da-random_rotation-order': 2,
 'da-rescaling-activate': False,
 'da-rescaling-factor_max': 1.2,
 'da-rescaling-order': 1,
 'da-shifting-activate': True,
 'da-shifting-order': 0,
 'da-shifting-percentage_max': 0.1,
 'da-type': 'all',
 'dataset_mean': 120.95,
 'dataset_variance': 60.23,
 'd

We create variables to be used by the network when training as well as the directory to store the model. If the directory already exists we add a timestamp at the end of the directory name.

In [6]:
path_model = os.path.join('../models/', dir_model_save)
if os.path.exists(path_model): #If the path exists already we add a timestamp at the end of dir name.
    dir_model_save += '_' + str(int(time.time()))[-4:]
    path_model = os.path.join('../models/', dir_model_save)
os.makedirs(path_model)

We then make sure that the config file is valid by using a function we imported. __update_config__ updates the first config (default_configuration()) with the second (config_network) so that the parameters in config_network are at least the parameters needed by the network to run.

In [7]:
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config)

Management of the config file: if it already exists, we load it, else, we write it.

In [8]:
path_config_file=os.path.join(path_model,'config_network.json')
if os.path.exists(path_config_file): # if there is already a configfile
    with open(path_config_file, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_config_file, 'w') as f:
        json.dump(config, f, indent=2)
        config_network=config
   
print 'Chosen parameters :\n'
for param_name, param_value in config_network.iteritems():
    print param_name, ' - ', param_value

Chosen parameters :

trainingset_patchsize  -  512
thresholds  -  [0, 0.2, 0.8]
learning_rate_decay_type  -  polynomial
convolution_per_layer  -  [3, 3, 3, 3]
da-gaussian_blur-sigma_max  -  1.5
da-shifting-percentage_max  -  0.1
da-shifting-activate  -  True
batch_norm_decay_decay_activate  -  True
da-flipping-activate  -  True
batch_norm_decay_starting_decay  -  0.7
n_classes  -  3
learning_rate_decay_activate  -  True
trainingset  -  SEM_3c_512
da-random_rotation-activate  -  False
da-rescaling-activate  -  False
batch_norm_decay_decay_period  -  72000
da-shifting-order  -  0
features_per_convolution  -  [[[1, 16], [16, 16], [16, 16]], [[16, 32], [32, 32], [32, 32]], [[32, 64], [64, 64], [64, 64]], [[64, 128], [128, 128], [128, 128]]]
batch_norm_decay_ending_decay  -  0.9
da-gaussian_blur-order  -  6
size_of_convolutions_per_layer  -  [[5, 5, 5], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
da-gaussian_blur-activate  -  True
dataset_mean  -  120.95
dilation_rate  -  [[1, 1, 1], [1, 1, 1], [1, 1,

We indicate to the network where to find our trainingset

In [ ]:
path_trainingset = os.path.join('../data/', config_network["trainingset"], 'training/')
path_model_init=None

Last part, but not the least: actually training the model!

In [ ]:
from AxonDeepSeg.train_network import train_model
train_model(path_trainingset, path_model, config_network, path_model_init=path_model_init,gpu='/gpu:0')

Total number of parameters to train: 125795
training start
Weighted cost selected
2018-01-14 13:13:54.378468-epoch:1-loss:0.9378167637463273-acc:0.5910731307391462
2018-01-14 13:14:39.783041-epoch:2-loss:0.8630186134371265-acc:0.6396939507846175
2018-01-14 13:15:24.838102-epoch:3-loss:0.8262268777551323-acc:0.6717092559255403
2018-01-14 13:16:10.481760-epoch:4-loss:0.8333541792014549-acc:0.6547117521022928
2018-01-14 13:16:56.302552-epoch:5-loss:0.8040626994494734-acc:0.6702900570014427
2018-01-14 13:17:41.816268-epoch:6-loss:0.8026461354617414-acc:0.6577420727959995
2018-01-14 13:18:27.075830-epoch:7-loss:0.7985694881143242-acc:0.6617052801724138
2018-01-14 13:19:12.697855-epoch:8-loss:0.7479385898031038-acc:0.6859945120482609
2018-01-14 13:19:58.171088-epoch:9-loss:0.7305560358639422-acc:0.6966964458597118
2018-01-14 13:20:43.659409-epoch:10-loss:0.6949410335770969-acc:0.7259770117957016
Best accuracy model saved in file: ../models/model_christian_0/best_acc_model.ckpt
Best loss mode

2018-01-14 14:08:45.515202-epoch:73-loss:0.45235559960891464-acc:0.8158041834831238
2018-01-14 14:09:30.940395-epoch:74-loss:0.46899719793221045-acc:0.8071519218642136
2018-01-14 14:10:16.521286-epoch:75-loss:0.471292527585194-acc:0.8094638914897523
Best accuracy model saved in file: ../models/model_christian_0/best_acc_model.ckpt
Best loss model saved in file: ../models/model_christian_0/best_loss_model.ckpt
2018-01-14 14:11:02.983411-epoch:76-loss:0.46990442378767605-acc:0.8079438004000433
2018-01-14 14:11:48.355394-epoch:77-loss:0.4494282438837248-acc:0.8192879228756345
2018-01-14 14:12:33.968169-epoch:78-loss:0.4264036355347469-acc:0.8269132396270489
2018-01-14 14:13:19.660060-epoch:79-loss:0.44589452805190255-acc:0.8130895700947991
2018-01-14 14:14:05.263774-epoch:80-loss:0.4528781864149817-acc:0.8155044008945597
Best accuracy model saved in file: ../models/model_christian_0/best_acc_model.ckpt
Best loss model saved in file: ../models/model_christian_0/best_loss_model.ckpt
2018-01

2018-01-14 15:09:38.633312-epoch:153-loss:0.4241729347870268-acc:0.8249397257278706
2018-01-14 15:10:24.447892-epoch:154-loss:0.41903244010333357-acc:0.8266645012230709
2018-01-14 15:11:09.996006-epoch:155-loss:0.421502287017888-acc:0.8267943242500568
Best accuracy model saved in file: ../models/model_christian_0/best_acc_model.ckpt
Best loss model saved in file: ../models/model_christian_0/best_loss_model.ckpt
2018-01-14 15:11:56.318947-epoch:156-loss:0.45221821295803993-acc:0.8085604741655548
2018-01-14 15:12:41.396150-epoch:157-loss:0.43508486192801904-acc:0.819073249553812
2018-01-14 15:13:26.989298-epoch:158-loss:0.4345459177576262-acc:0.8168167924058848
2018-01-14 15:14:12.690289-epoch:159-loss:0.4263067553783285-acc:0.824450394202923
2018-01-14 15:14:58.204593-epoch:160-loss:0.4258369998685245-acc:0.8216334355288538
2018-01-14 15:15:44.128554-epoch:161-loss:0.4341485510612356-acc:0.8190783755532627
2018-01-14 15:16:29.632455-epoch:162-loss:0.420163482427597-acc:0.827304182381465

2018-01-14 16:21:56.319930-epoch:248-loss:0.46388032313050886-acc:0.8049696540010387
